In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress INFO and WARNING messages
import argparse
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib
from sentence_transformers import SentenceTransformer

## train test split
from sklearn.model_selection import train_test_split


/home/dimitri/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Parameters
n_missing = 3
MODEL_PATH = "rnn_model.h5"
TOKENIZER_PATH = "tokenizer.pkl"
LABEL_ENCODER_PATH = "label_encoder.pkl"
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 128

In [3]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
splits = {'train': 'train.csv', 'validation': 'dev.csv', 'test': 'test.csv'}
df = pd.read_csv("hf://datasets/ibm-research/argument_quality_ranking_30k/" + splits["train"])

In [4]:
df.head()

,argument,topic,set,WA,MACE-P,stance_WA,stance_WA_conf
0,"""marriage"" isn't keeping up with the times. a...",We should abandon marriage,train,0.846165,0.297659,1,1.000000
1,.a multi-party system would be too confusing a...,We should adopt a multi-party system,train,0.891271,0.726133,-1,1.000000
2,`people reach their limit when it comes to the...,Assisted suicide should be a criminal offence,train,0.730395,0.225212,-1,1.000000
3,"100% agree, should they do that, it would be a...",We should abolish safe spaces,train,0.236686,0.004104,1,0.805517
4,A ban on naturopathy creates a cohesive front ...,We should ban naturopathy,train,0.753805,0.337724,1,1.000000


In [5]:
argument = list(df.argument)
topic = list(df.topic)
note = list(df.WA)

In [12]:
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
PADDINGS_TOPICS = 10
PADDINGS_ARGUMENTS = 100

nltk.download('punkt')

data = []
for i in range(len(argument)):
    arg_token = word_tokenize(argument[i].lower())
    arg_topic = word_tokenize(topic[i].lower())
    if len(arg_topic) < PADDINGS_TOPICS:
        arg_topic += ['<pad>'] * (PADDINGS_TOPICS - len(arg_topic))
    if len(arg_token) < PADDINGS_ARGUMENTS:
        arg_token += ['<pad>'] * (PADDINGS_ARGUMENTS - len(arg_token))

    data.append(arg_topic + arg_token)

model_embedding = Word2Vec(sentences=data, vector_size=100, window=5, min_count=1, workers=4)

print("Exemple de vocabulaire :", list(model_embedding.wv.key_to_index.keys())[:10])

[nltk_data] Downloading package punkt to /home/dimitri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Exemple de vocabulaire : ['<pad>', 'should', 'we', 'the', 'to', '.', 'and', 'of', 'a', 'be']


In [13]:
import numpy as np

corpus = [
    "Deep learning is powerful",
    "Neural networks are used for computer vision",
    "Word2Vec learns word representations"
]

def sentence_to_vec(sentence, model):
    vectors = [model.wv[word] for word in sentence if word in model.wv]
    return np.array(vectors)

In [14]:
X = np.array([sentence_to_vec(sent, model_embedding) for sent in data])

In [11]:
X[0].shape

(210, 100)

In [15]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, np.array(list(note)), test_size=0.2, random_state=42)

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Bidirectional, LSTM
# Construire le modèle LSTM avec les embeddings de BERT
model = Sequential()

# Ajouter des couches bidirectionnelles LSTM
model.add(Bidirectional(LSTM(128, return_sequences=True, input_shape=(MAX_SEQUENCE_LENGTH, X.shape[-1]))))
model.add(Bidirectional(LSTM(64)))

# Ajouter des couches denses
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))

# Compiler le modèle
model.compile(loss='mean_absolute_error', optimizer='adam')

In [19]:
# Train and evaluate model with train and test
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
525/525 [==============================] - 84s 153ms/step - loss: 0.1511 - val_loss: 0.1495
Epoch 2/10
525/525 [==============================] - 90s 171ms/step - loss: 0.1397 - val_loss: 0.1390
Epoch 3/10
525/525 [==============================] - 86s 164ms/step - loss: 0.1370 - val_loss: 0.1367
Epoch 4/10
525/525 [==============================] - 72s 137ms/step - loss: 0.1342 - val_loss: 0.1350
Epoch 5/10
525/525 [==============================] - 70s 134ms/step - loss: 0.1329 - val_loss: 0.1356
Epoch 6/10
103/525 [====>.........................] - ETA: 54s - loss: 0.1283

KeyboardInterrupt: 

In [32]:
def predict(data):
    data = [data]
    # Tokenisation et encodage des textes
    encoded_inputs = tokenizer(data, padding=True, truncation=True, max_length=MAX_SEQUENCE_LENGTH, return_tensors="pt")
    # Obtenir les embeddings de BERT
    with torch.no_grad():
        outputs = bert_model(**encoded_inputs)
        # Utiliser les embeddings de la dernière couche cachée
        bert_embeddings = outputs.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)

    # Convertir les embeddings en numpy pour Keras
    bert_embeddings = bert_embeddings.numpy()
    print(bert_embeddings.shape)
    prediction = model.predict(bert_embeddings)
    return prediction

In [ ]:
arg = "a zero tolerance policy means that parents would give complete control of discipline to the school without any regard for family morals and teachings."
top = "We should adopt a zero-tolerance policy in schools"
predict(arg + top)

(1, 22, 768)
1/1 [==============================] - 1s 689ms/step


array([[0.7031841]], dtype=float32)